# Installing Textblob

In [ ]:
!pip install -U textblob

# Downloading corpora

In [ ]:
!python -m textblob.download_corpora

# Importing Libraries

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import re
import string
import nltk
import spacy
import gensim
from spacy import displacy
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.tokenize import WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.stem import WordNetLemmatizer, PorterStemmer, SnowballStemmer
from nltk.tokenize import word_tokenize
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()
%matplotlib inline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
import numpy as np
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer,TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from nltk.probability import FreqDist
from sklearn.svm import LinearSVC
from textblob import TextBlob
from gensim.models import Word2Vec

# Loading Data from CSV to DataFrame

In [2]:
reviews_df = pd.read_csv('C:/Users/SANA JALGAONKAR/Desktop/Text Analysis/Amazon indian product reviews.csv', encoding="UTF-8")
reviews_df.head()#path of the csv file

,asin,name,date,rating,review
0,B07W7CTLD1,Mamaearth-Onion-Growth-Control-Redensyl,2019-09-06,1,I bought this hair oil after viewing so many g...
1,B07W7CTLD1,Mamaearth-Onion-Growth-Control-Redensyl,2019-08-14,5,Used This Mama Earth Newly Launched Onion Oil ...
2,B07W7CTLD1,Mamaearth-Onion-Growth-Control-Redensyl,2019-10-19,1,So bad product...My hair falling increase too ...
3,B07W7CTLD1,Mamaearth-Onion-Growth-Control-Redensyl,2019-09-16,1,Product just smells similar to navarathna hair...
4,B07W7CTLD1,Mamaearth-Onion-Growth-Control-Redensyl,2019-08-18,5,I have been trying different onion oil for my ...


# check the dimensionality of the dataframe

In [3]:
reviews_df.shape

(2782, 5)

# Checking the null Columns

In [4]:
# any null columns
reviews_df.isnull().sum()

asin      0
name      0
date      0
rating    0
review    4
dtype: int64

#### In the the review column, four rows without review text, we drop the rows with the null columns

In [5]:
reviews_df = reviews_df.dropna()
#resetting the index
reviews_df = reviews_df.reset_index(drop=True)
# any null columns
reviews_df.isnull().sum()

asin      0
name      0
date      0
rating    0
review    0
dtype: int64

# Preprocessing

In [6]:
# remove all characters not number or characters
def cleanText(input_string):
    modified_string = re.sub('[^A-Za-z0-9]+', ' ', input_string)
    return(modified_string)
reviews_df['review'] = reviews_df.review.apply(cleanText)
reviews_df['review'][150]

'I am writing this review after using it around 20 days It seems very natural and chemical free and is very gentle on skin But it does its job of cleaning the skin properly It contains tea tree which is one of my favourite ingredients for skin care Give it a try its definitely better than all other chemicals containing face washes And its even affordable as compared to other natural brands available in market '

# Extracting the brand name

In [7]:
# From the name we extract the brand
reviews_df['brandName'] = reviews_df['name'].str.split('-').str[0]
reviews_df.head()

,asin,name,date,rating,review,brandName
0,B07W7CTLD1,Mamaearth-Onion-Growth-Control-Redensyl,2019-09-06,1,I bought this hair oil after viewing so many g...,Mamaearth
1,B07W7CTLD1,Mamaearth-Onion-Growth-Control-Redensyl,2019-08-14,5,Used This Mama Earth Newly Launched Onion Oil ...,Mamaearth
2,B07W7CTLD1,Mamaearth-Onion-Growth-Control-Redensyl,2019-10-19,1,So bad product My hair falling increase too mu...,Mamaearth
3,B07W7CTLD1,Mamaearth-Onion-Growth-Control-Redensyl,2019-09-16,1,Product just smells similar to navarathna hair...,Mamaearth
4,B07W7CTLD1,Mamaearth-Onion-Growth-Control-Redensyl,2019-08-18,5,I have been trying different onion oil for my ...,Mamaearth


In [8]:
reviews_df['brandName'].value_counts()

Mamaearth      200
Godrej         200
Titan          200
Himalaya       200
Cinthol        200
Streax         188
Society        182
Tata           180
Dettol         180
Amul           164
Fastrack       160
Mysore         154
Patanjali      152
Britannia      120
Paper           40
MYSORE          40
Reflex          40
Natural         40
Maaza           20
Glucon          20
Coca            20
Savlon          20
Maggi           20
PATANJALI       20
NutriChoice     12
Indiana          6
Name: brandName, dtype: int64

In [9]:
reviews_df['brandName'] = reviews_df['brandName'].str.title()
reviews_df.brandName.unique()

array(['Mamaearth', 'Godrej', 'Titan', 'Maaza', 'Paper', 'Indiana',
       'Coca', 'Natural', 'Maggi', 'Glucon', 'Amul', 'Patanjali',
       'Dettol', 'Savlon', 'Cinthol', 'Britannia', 'Nutrichoice',
       'Streax', 'Himalaya', 'Society', 'Tata', 'Fastrack', 'Reflex',
       'Mysore'], dtype=object)

# Extracting the Product name

In [10]:
# Extracting the product from the name column
products = []
for value in reviews_df['name']:
    indx = len(value.split('-')[0])+1
    products.append(value[indx:])
reviews_df['product'] = products
reviews_df['product'].unique()

array(['Onion-Growth-Control-Redensyl',
       'Percent-Natural-Berry-Toothpaste',
       'Natural-Turmeric-Saffron-brightning',
       'Illuminate-Vitamin-Radiant-Turmeric',
       'Blemishes-Pigmentation-Blemish-Mulberry', 'Face-Wash-100-ml',
       'Moisturizing-Baby-Bathing-Oatmeal',
       'Protekt-Master-Blaster-Handwash', 'No-1-Bathing-Soap-Lime',
       'No-1-Bathing-Soap-Turmeric', 'Direct-Cool-Refrigerator-1905-PTDI',
       'aer-Pocket-Bathroom-Fragrance',
       'Security-Solutions-SEEC9060-Electronic',
       'Matic-Spray-Violet-Valley',
       'Security-Solutions-Goldilocks-Personal',
       'ViroShield-Disinfecting-VIROSHIELD-30UV',
       'Analog-Black-Dial-Watch-1805NM01',
       'Analog-White-Dial-Watch-NK1639SL03',
       'Karishma-Analog-Blue-Watch-1774SM01',
       'Karishma-Analog-Black-Watch-NK1639SM02',
       'Octane-Analog-Silver-Watch-NK1650BM03',
       'Analog-Black-Dial-Watch-NK1730SL02',
       'Karishma-Analog-Champagne-Watch-NK1580YL05',
       'Analog-

In [11]:
reviews_df.head()

,asin,name,date,rating,review,brandName,product
0,B07W7CTLD1,Mamaearth-Onion-Growth-Control-Redensyl,2019-09-06,1,I bought this hair oil after viewing so many g...,Mamaearth,Onion-Growth-Control-Redensyl
1,B07W7CTLD1,Mamaearth-Onion-Growth-Control-Redensyl,2019-08-14,5,Used This Mama Earth Newly Launched Onion Oil ...,Mamaearth,Onion-Growth-Control-Redensyl
2,B07W7CTLD1,Mamaearth-Onion-Growth-Control-Redensyl,2019-10-19,1,So bad product My hair falling increase too mu...,Mamaearth,Onion-Growth-Control-Redensyl
3,B07W7CTLD1,Mamaearth-Onion-Growth-Control-Redensyl,2019-09-16,1,Product just smells similar to navarathna hair...,Mamaearth,Onion-Growth-Control-Redensyl
4,B07W7CTLD1,Mamaearth-Onion-Growth-Control-Redensyl,2019-08-18,5,I have been trying different onion oil for my ...,Mamaearth,Onion-Growth-Control-Redensyl


# Reviewing counts and brands

#### Installing Chart_studio

In [ ]:
!pip install chart-studio

#### Installing Cufflinks

In [ ]:
!pip install cufflinks

In [ ]:
import plotly.graph_objs as go
import chart_studio.plotly as py
import cufflinks
pd.options.display.max_columns = 30
from IPython.core.interactiveshell import InteractiveShell
import plotly.figure_factory as ff
InteractiveShell.ast_node_interactivity = 'all'
from plotly.offline import iplot
cufflinks.go_offline()
cufflinks.set_config_file(world_readable=True, theme='pearl')
reviews_df.groupby(['brandName']).count()['review'].iplot(kind='bar',yTitle='Count',linecolor='black',opacity=0.8,
                                                         title="Brands",xTitle="Brands")

# Text analysis with NLTK and Vader Sentiment analyzer

### Text Preprocessing

#### converting Review data into lower case

In [ ]:
#converting to lower case
reviews_df['clean_review_text']=reviews_df['review'].str.lower()

#### Removing punctuations

In [ ]:
#removing punctuations
reviews_df['clean_review_text']=reviews_df['clean_review_text'].str.translate(str.maketrans('','',string.punctuation))

#### Removing stopwords

In [ ]:
stopWords=stopwords.words('english')+['the', 'a', 'an', 'i', 'he', 'she', 'they', 'to', 'of', 'it', 'from']
def removeStopWords(stopWords, rvw_txt):
    newtxt = ' '.join([word for word in rvw_txt.split() if word not in stopWords])
    return newtxt
reviews_df['clean_review_text'] = [removeStopWords(stopWords,x) for x in reviews_df['clean_review_text']]

#### Splitting text into words

In [ ]:
#splitting text into words
tokenList=[]
for indx in range(len(reviews_df)):
       token=word_tokenize(reviews_df['clean_review_text'][indx])
       tokenList.append(token)
reviews_df['review_tokens'] = tokenList
reviews_df.head()

#### Downloading the vader_lexicon Package

In [ ]:
nltk.download('vader_lexicon')

#### Initializing the sentiment_model

In [ ]:
sentiment_model = SentimentIntensityAnalyzer()
sentiment_scores=[]
sentiment_score_flag = []
for text in reviews_df['clean_review_text']:
        sentimentResults = sentiment_model.polarity_scores(text)
        sentiment_score = sentimentResults["compound"]
        #print(sentimentResults)
        #The compound value reflects the overall sentiment ranging from -1 being very negative and +1 being very positive.
        sentiment_scores.append(sentiment_score)
        # marking the sentiments as positive, negative and neutral 
        if sentimentResults['compound'] >= 0.05 : 
            sentiment_score_flag.append('positive')
  
        elif sentimentResults['compound'] <= - 0.05 : 
            sentiment_score_flag.append('negative')
  
        else : 
            sentiment_score_flag.append('neutral')
            
reviews_df['scores']=sentiment_scores
reviews_df['scoreStatus'] = sentiment_score_flag

In [ ]:
reviews_df.head()

### Wordcloud displaying top words 

In [ ]:
def show_wordcloud(data, title = None):
    wordcloud = WordCloud(
        background_color = 'white',
        max_words = 200,
        max_font_size = 40, 
        scale = 3,
        random_state = 42
    ).generate(str(data))

    fig = plt.figure(1, figsize = (20, 20))
    plt.axis('off')
    if title: 
        fig.suptitle(title, fontsize = 20)
        fig.subplots_adjust(top = 2.3)

    plt.imshow(wordcloud)
    plt.show()
    
# print wordcloud
show_wordcloud(reviews_df["clean_review_text"])

In [ ]:
# print wordcloud for positive review
positiveReviews_df =reviews_df.loc[reviews_df['scoreStatus'] == "positive"]
show_wordcloud(positiveReviews_df["clean_review_text"])

In [ ]:
# print wordcloud for negative reviews
negativeReviews_df =reviews_df.loc[reviews_df['scoreStatus'] == "negative"]
show_wordcloud(positiveReviews_df["clean_review_text"])

In [ ]:
positiveReviews_df.groupby(['brandName']).count()['scoreStatus'].iplot(kind='bar',yTitle='Count',linecolor='black',opacity=0.8,
                                                         title="Brands",xTitle="Brands")

In [ ]:
negativeReviews_df.groupby(['brandName']).count()['scoreStatus'].iplot(kind='bar',yTitle='Count',linecolor='black',opacity=0.8,
                                                         title="Brands",xTitle="Brands")

#### Checking Indiana Brandname

In [ ]:
a = pd.DataFrame(positiveReviews_df.groupby(['brandName']).count()["clean_review_text"])
positiveReviews_df.loc[positiveReviews_df['brandName'] == "Indiana"]["review"]

In [ ]:
b = pd.DataFrame(negativeReviews_df.groupby(['brandName']).count()["clean_review_text"])

#### The 6 reviews for “Indiana” brand are as below and we see them as positive.

In [ ]:
c = pd.concat([a, b], axis=1)
c.columns = ['positiveReviews','negativeReviews']
c

## TF and TF-IDF and Bag of Words

To represent our text numerically we have the Bag of Words model like TF, TFIDF

TF-IDF aims to quantify the importance of a given word relative to other words in the document and in the corpus.

In [ ]:
features = CountVectorizer()
features.fit(reviews_df["clean_review_text"])
print(len(features.vocabulary_))
print(features.vocabulary_)

BoW is a classical text representation technique. The text under consideration as a collection of words while ignoring the order and context. BoW converts text into the matrix of occurrence of words within a given document. It focuses on whether given words occurred or not in the document, and it generates a matrix that we might see referred to as a BoW matrix or a document term matrix.

In [ ]:
bagofWords = features.transform(reviews_df["clean_review_text"])
print(bagofWords)

In [ ]:
print(bagofWords.toarray())

In [ ]:
print(features.get_feature_names())

# Splitting the data

In [ ]:
df = pd.concat([positiveReviews_df,negativeReviews_df])
df = df[["clean_review_text","scoreStatus"]]
df['scoreStatus'] = (df['scoreStatus'] == 'positive')*1
X = df["clean_review_text"]
y = df["scoreStatus"]
X_train, X_test, y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=0)

In [ ]:
X_train = features.fit_transform(X_train)
X_test = features.transform(X_test)

# Logistic Regression for training the classification model

In [ ]:
#k fold cross validation with k=5
scores = cross_val_score(LogisticRegression(),X_train,y_train,cv=5)
print(np.mean(scores))

In [ ]:
#Logistic Model building
model=LogisticRegression()
model.fit(X_train,y_train)
print(model.score(X_train,y_train))
print(model.score(X_test,y_test))

In [ ]:
#confusion Matrix
y_pred = model.predict(X_test)
confusion_matrix(y_test,y_pred)

# How the trained model works on new data

In [ ]:
text = "the product great"
model.predict(features.transform([text]))[0]

In [ ]:
text = "bad"
model.predict(features.transform([text]))[0]

In [ ]:
text = "sucks"
model.predict(features.transform([text]))[0]

In [ ]:
text = "very bad"
model.predict(features.transform([text]))[0]

In [ ]:
text = "not good"
model.predict(features.transform([text]))[0]

great and bad rightly predicted but the model wrongly predicts “sucks” and “not good”

# Frequency Distribution

In [ ]:
tokenized_word=word_tokenize((reviews_df['clean_review_text'].to_string()))
#Frequency Distribution
fdist = FreqDist(tokenized_word)
# Frequency Distribution Plot
fdist.plot(30,cumulative=False)
plt.show()
'='

# Text Analytics with spacy

In [ ]:
nlp=spacy.load("en_core_web_sm")

In [ ]:
text = reviews_df['review'][1130]
doc=nlp(text)
type(doc)
print(doc)

#### Tokenizing the Text

In [ ]:
#Tokens
for token in doc:
    print(token.text)

#### Stopwords

In [ ]:
#Stopwords
stopwords=spacy.lang.en.stop_words.STOP_WORDS
stopWords = list(stopwords)
len(stopWords)

In [ ]:
for token in doc:
    if token.is_stop == False:
        print(token)

#### Lemmatization

Finding the roots of all the words using spaCy lemmatization.

In [ ]:
print(len(doc))
doc2=[]
for token in doc:
    if not token.is_stop:
        doc2.append(token)
print(len(doc2))

In [ ]:
#lemmatization
for review_text in doc:
    print(review_text.text,review_text.lemma_)

In [ ]:
#POS
for token in doc:
   print(token,token.tag_,token.pos_,spacy.explain(token.tag_))

In [ ]:
displacy.render(doc,style='dep',jupyter=True,options={'distance':90})

In [ ]:
for entity in doc.ents:
    print(entity.text,'---->',entity.label_)
displacy.render(doc,style='ent',jupyter=True)

## Downloaded the large pre trained model

In [ ]:
#word vectors and similarity
#large pre trained model
!python -m spacy download en_core_web_lg

In [ ]:
import en_core_web_lg
nlp = en_core_web_lg.load()
doc=nlp(text)
for token in doc:
    print(token.text,'---->',token.has_vector)

In [ ]:
for token in doc:
    print(token.text,'',token.vector_norm)

### Checking the Similarity score

In [ ]:
#similarity score
text="eat"
text1="ate"
doc=nlp(text)
doc1=nlp(text1)
doc.similarity(doc1)

In [ ]:
#similarity score
text="good"
text1="bad"
doc=nlp(text)
doc1=nlp(text1)
doc.similarity(doc1)

In [ ]:
#similarity score
text="hot"
text1="summer"
doc=nlp(text)
doc1=nlp(text1)
doc.similarity(doc1)

In [ ]:
#similarity score
text="excellent"
text1="good"
doc=nlp(text)
doc1=nlp(text1)
doc.similarity(doc1)

In [ ]:
#similarity score
text="sucks"
text1="bad"
doc=nlp(text)
doc1=nlp(text1)
doc.similarity(doc1)

### Combining positive and negative datasets to build the training data for training a classification model

In [ ]:
df = pd.concat([positiveReviews_df,negativeReviews_df])
df = df[["clean_review_text","scoreStatus"]]
df['scoreStatus'] = (df['scoreStatus'] == 'positive')*1

In [ ]:
#Tokenization
punct = string.punctuation
print(punct)
def cleanText(sent):
    doc = nlp(sent)
    tokens = []
    for token in doc:
        if token.lemma != "-PRON-":
            tokens.append(token.lemma_.lower().strip())
        else:
            tokens.append(token.lemma_)
            
    cleanTokens = []
    for token in tokens:
        if token not in stopWords and token not in punct:
            cleanTokens.append(token)
    return cleanTokens

In [ ]:
#TFIDF
tfidf = TfidfVectorizer(tokenizer = cleanText)
classifier = LinearSVC()
X = df["clean_review_text"]
y = df["scoreStatus"]
X_train, X_test, y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=0)

### Training the model with SVM

In [ ]:
X_train.shape, X_test.shape

In [ ]:
clf = Pipeline([('tfidf',tfidf),('clf',classifier)])
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)

In [ ]:
print(classification_report(y_test,y_pred))

In [ ]:
confusion_matrix(y_test,y_pred)

### Analyzing with TextBlob

polarity varies from -1 (negative) to +1 (positive)

subjectivity varies from 0 to 1

In [ ]:
nltk.download('averaged_perceptron_tagger')
blob = TextBlob(str(reviews_df['review']))
pos_df = pd.DataFrame(blob.tags, columns = ['word' , 'pos'])
pos_df = pos_df.pos.value_counts()[:20]
pos_df.iplot(
    kind='bar',
    xTitle='POS',
    yTitle='count', 
    title='Top 20 Part-of-speech tagging for review corpus')

In [ ]:
pol = lambda x: TextBlob(x).sentiment.polarity
sub = lambda x: TextBlob(x).sentiment.subjectivity
reviews_df["polarity"] = reviews_df["review"].apply(pol)
reviews_df["subjectivity"] = reviews_df["review"].apply(sub)

In [ ]:
#distribution of rating
sns.countplot(x='rating', data=reviews_df)

In [ ]:
reviews_df.head()

In [ ]:
print("negative reviews")
most_negative = reviews_df[reviews_df.polarity == -1].review.head()
print(most_negative)
print("positive reviews")
most_positive = reviews_df[reviews_df.polarity == 1].review.head()
print(most_positive)

### Text analysis with gensim and word2vec

#### Word2Vec

Word2Vec actually takes the semantic meaning of the words and their relationships between other words. It learns all the internal relationships between the words.It represents the word in dense vector form.

In [ ]:
sentences = reviews_df['review_tokens'][10:20]
sentences

Using Gensim’s library we have Word2Vec which takes parameters like min_count = 1 considers only if word repeats more than 1 time in entire data.

In [ ]:
#train model
model = Word2Vec(sentences, min_count=1)
print(model)

In [ ]:
#vocab
words=list(model.wv.vocab)
print(words)

In [ ]:
Downloading PUNKT

In [ ]:
nltk.download('punkt')

In [ ]:
reviewsText = reviews_df.clean_review_text.values
reviewsVec = [nltk.word_tokenize(review) for review in reviewsText]
len(reviewsVec)

#### Using the trained model to find the similarity of certain word.

In [ ]:
model = Word2Vec(reviewsVec,min_count=1, size=32)
model.most_similar('soothing')

In [ ]:
model = Word2Vec(reviewsVec,min_count=1, size=32)
model.most_similar('packaging')